In [4]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp_2022/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['mar']

In [5]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil

In [6]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [7]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство'))]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[['inn_x', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [8]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [9]:
def load_groups(file):
    df = pd.read_excel(path_to_docs + file + '.xlsx', sheet_name='УГД')
    ped = df[df['Педагогический'] == 1]['Должность'].tolist()
    isp = df[df['Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми'] == 1]['Должность'].tolist()
    aup = df[df['УГД'] == 'Административно-управленческий персонал']['Должность'].to_list()
    return ped, isp, aup

In [10]:
gku = ["ГБУ МГДУ","ГАОУ ДПО ЦПМ","ГАОУ ДПО МЦКО","ГКУ ЦФО ДОНМ","ГБУ ГППЦ ДОНМ","ГКУ СФК ДОНМ","ГКУ Дирекция ДОНМ","ГБОУ ДПО МЦПС","ГБОУ ГМЦ ДОНМ","ГКУ Дирекция по строительству и реконструкции ДОНМ","ГАОУ ДПО ""Корпоративный университет""","ГАУ ""Центр цифровизации образования""","ГАУ Медиацентр","ГАОУ ДПО МЦРПО"]
gku_inn = [7702061938, 7725618950, 7725539709, 9718071371, 7726317748, 7704191153, 9705101759, 7719210793, 7705399348, 7705020295, 7714239823, 7727190237, 7718924940, 7707329480]

In [19]:
ppl = load_data(months[-1])
ped, isp, aup = load_groups('input')
ppl = ppl[(ppl['job_' + months[-1]].isin(aup + ped) == False) & (ppl['sum_' + months[-1]] > 200000) & (ppl['inn'].isin(gku_inn) == False)]
print_df(ppl, '200+_' + months[-1])

In [16]:
otv = pd.read_excel(path_to_docs + 'Ответственные.xlsx').rename(columns={'инн орг': 'inn', 'округ': 'okr', 'Зарпл. Обслуживание': 'otv', 'Наименование': 'sch_name'})
otv = otv[['inn', 'okr', 'otv', 'sch_name']]
otv

,inn,okr,otv,sch_name
0,7743085543,САО,Рагимова Виктория Олеговна,ГБОУ Спецшкола № 1
1,7713229928,САО,Горбанева Светлана Васильевна,ГКОУ КШИ № 1
2,7704271031,ЦАО,Майильян Елена Александровна,ГБПОУ ПТ № 2
3,7714093099,САО,Горбанева Светлана Васильевна,ГКОУ СКОШИ № 2
4,7709438340,ЦАО,Майильян Елена Александровна,ГБПОУ КСУ № 3
...,...,...,...,...
935,7728113806,NaN,NaN,ИКИ РАН
936,7728013512,NaN,NaN,ИПУ РАН
937,7717067481,NaN,NaN,ГБОУ ШОР № 1
938,7729133548,NaN,NaN,СУНЦ МГУ


In [14]:
fio = pd.read_excel(path_to_docs + 'ФИО+СНИЛС.xlsx').rename(columns={'СНИЛС': 'snils', 'ФИО': 'fio'})
fio

,snils,fio
0,134-527-811 55,Аиткулова Ильмира Зарифовна
1,209-532-112 40,Шахова Наталья Николаевна
2,132-177-301 25,Трясина Дина Львовна
3,070-440-299 32,Салихова Ольга Васильевна
4,009-646-095 65,Харькова Оксана Петровна
...,...,...
185261,134-606-800 44,Скрипель Александр Фёдорович
185262,181-087-647 80,Петрова Екатерина Игоревна
185263,121-018-792 09,Блохина Мария Владимировна
185264,154-297-412 77,Куликова Елена Сергеевна


In [20]:
pretty_ppl = pd.merge(ppl, fio, how='left', on='snils')
pretty_ppl = pd.merge(pretty_ppl, otv, how='left', on='inn')
pretty_ppl

,inn,snils,sum_mar,stv_mar,job_mar,fio,okr,otv,sch_name
0,5003021368,015-826-692 62,218670.00,1.00,Контрактный управляющий,Фурсова Анна Аркадьевна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 1788
1,5003021368,143-108-314 14,213670.81,1.00,Ведущий экономист,ШЕЛАМОВА ЮЛИЯ СЕРГЕЕВНА,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 1788
2,5003021495,106-994-733 95,233574.85,1.00,Ведущий экономист,Кваснюк Анастасия Васильевна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 2070
3,5003021640,075-365-019 69,262300.00,1.00,Ведущий экономист,Васина Виктория Викторовна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 1392 им. Д.В. Рябинкина
4,5003096290,059-350-413 62,295015.00,0.90,Ведущий экономист,Пестова Юлия Владимировна,ТиНАО,Горбанева Светлана Васильевна,ГБОУ Школа № 2065
...,...,...,...,...,...,...,...,...,...
460,7743905443,187-737-025 14,220686.67,1.00,Специалист,Кашницкая Елизавета Александровна,САО,Рагимова Виктория Олеговна,ГБОУ Школа № 2098 имени Героя Советского Союза...
461,7743940590,064-656-278 88,207005.42,1.00,Ведущий экономист,Лапшинова Юлия Андреевна,САО,Рагимова Виктория Олеговна,ГБОУ Школа Перспектива
462,7743940590,197-413-167 92,216396.36,1.00,Ведущий специалист,Боровой Андрей Станиславович,САО,Рагимова Виктория Олеговна,ГБОУ Школа Перспектива
463,9715206976,056-748-271 96,211313.66,1.00,Заведующий учебной частью,Пьянкова Татьяна Александровна,СВАО,Горбанева Светлана Васильевна,ГБОУ Школа Марьина Роща


In [21]:
print_df(pretty_ppl[['sch_name', 'okr', 'inn', 'fio', 'job_' + months[-1], 'sum_' + months[-1], 'stv_' + months[-1], 'otv']], '200+_' + months[-1] + '_cute')

In [ ]:
aup

['Главный бухгалтер',
 'Директор',
 'Директор филиала',
 'Заведующий',
 'Заместитель главного бухгалтера',
 'Заместитель директора',
 'Заместитель начальника',
 'Заместитель руководителя',
 'Начальник',
 'Первый заместитель начальника',
 'Первый заместитель руководителя',
 'Первый заместитель директора',
 'Заместитель начальника - начальник управления',
 'Первый проректор',
 'Президент',
 'Проректор',
 'Ректор',
 'Руководитель']